In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm
from statistics import mean, variance
from functools import lru_cache
from math import sqrt
from typing import List, Tuple
import itertools
import json
from IPython.display import clear_output
import os
import time

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
#Initializing parameters
input_shape = (28, 28, 1)
#batch_size = 64
#num_classes = 10
#epochs = 5

#Cleaning data
x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test=x_test/255.0

y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

2025-01-13 10:08:24.755405: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def make_model(
    cnn_layer_sizes : List[Tuple[int, Tuple[int, int]]],
    dense_layer_sizes : List[int]
):
    activation_function = 'relu'
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(input_shape))
    for i in range(len(cnn_layer_sizes)-1):
        model.add(tf.keras.layers.Conv2D(cnn_layer_sizes[i][0], cnn_layer_sizes[i][1], activation=activation_function))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    if len(cnn_layer_sizes) > 0:
        model.add(tf.keras.layers.Conv2D(cnn_layer_sizes[-1][0], cnn_layer_sizes[-1][1], activation=activation_function))
    model.add(tf.keras.layers.Flatten())
    for elem in dense_layer_sizes:
        model.add(tf.keras.layers.Dense(elem, activation = activation_function))
    model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])
    return model

In [5]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.995):
            print("\nReached 99.5% accuracy so cancelling training!")
            self.modelel.stop_training = True

"""callbacks = myCallback()

model = make_model([(10, (5,5)), (5, (3,3))], [64, 64], 'tanh')

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    validation_split=0.1,
                    callbacks=[callbacks])"""

"callbacks = myCallback()\n\nmodel = make_model([(10, (5,5)), (5, (3,3))], [64, 64], 'tanh')\n\nhistory = model.fit(x_train, y_train,\n                    batch_size=batch_size,\n                    epochs=5,\n                    validation_split=0.1,\n                    callbacks=[callbacks])"

In [6]:
#CONFIGURATIONS
cnn_layer_depth = [0,1]
cnn_layer_dimensions = [(3,3), (4,4), (5,5)]
num_cnn_layers = [1,2,3]
num_dense_layers = [1,2,3]
dense_layer_sizes = [64, 128, 256]
batch_size = 64

def is_decreasing(L : List[int]):
    prev = L[0]
    for i in range(1, len(L)):
        if L[i] < prev:
            return False
    return True

def gen_cnn_layer_combinations():
    L = []
    for layer in num_cnn_layers:
        for dim in cnn_layer_dimensions:
            L.append((layer, dim))
    return L

def send_model_to_github(filename):
    os.system(f"git add {filename}")
    os.system(f"git commit -m \"Added {filename}\"")
    os.system(f"git push origin main")
    os.system(f"rm -r {filename}")
    return
    
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.995):
            print("\nReached 99.5% accuracy so cancelling training!")
            self.modelel.stop_training = True

def iterate_combinations(starter):
    counter = 0
    for cnn_depth in cnn_layer_depth:
        for cnn_layer_dim in itertools.product(gen_cnn_layer_combinations(), repeat=cnn_depth):
            for dense_depth in num_dense_layers:
                for dense_layers in itertools.product(dense_layer_sizes, repeat=dense_depth):
                    if counter > starter and is_decreasing(dense_layers):
                        callbacks = myCallback()
                        model = make_model(list(cnn_layer_dim), list(dense_layers))
                        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=4,
                            validation_split=0.1,
                            callbacks=[callbacks])
                        filename = f'model_configs/model{counter}_savefile'
                        model.save(filename)
                        
                        yield model, str([cnn_layer_dim]), str([dense_layers]), batch_size, filename, history.history['acc'][-1], filename
                    else:
                        yield tf.keras.models.Sequential(), "", "", 0, "", 0, ""
                    counter += 1
                    
    return 

#iterate_combinations()

In [7]:
counter = 0
for cnn_depth in cnn_layer_depth:
    for cnn_layer_dim in itertools.product(gen_cnn_layer_combinations(), repeat=cnn_depth):
        for dense_depth in num_dense_layers:
            for dense_layers in itertools.product(dense_layer_sizes, repeat=dense_depth):
                counter += is_decreasing(dense_layers)
counter

230

In [28]:
def create_random_screen(eps = 0.01):
    return (np.random.rand(28,28) * 2 * eps) - eps

def selection(scores):
    #https://www.geeksforgeeks.org/python-indices-of-n-largest-elements-in-list/
    return sorted(range(len(scores)), key = lambda sub: scores[sub])[-25:]

def make_modifications(screen, randomness=0.001, eps=0.01):
    new_screen = screen + (np.random.rand(28,28) - 0.5) * 2 * randomness
    for i in range(28):
        for j in range(28):
            if new_screen[i][j] > eps:
                new_screen[i][j] = eps
            elif new_screen[i][j] < -eps:
                new_screen[i][j] = -eps
    return new_screen

def random_indices(n, total):
    L = [True] * n + [False] * (total - n)
    random.shuffle(L)
    return L

def sample(n, screens):
    index = random_indices(n, len(screens))
    return np.array([screens[i] for i in range(len(screens)) if index[i]])

def find_mean_var(m):
    l = m.reshape(784)
    return mean(l), variance(l)

@lru_cache()
def get_these_values(from_value):
    return np.array([x_train[i] for i in range(len(x_train)) if y_train[i][from_value] == 1])

def mean_difference_initialization(target, from_value, eps):
    x_target = get_these_values(target)
    x_from_value = get_these_values(from_value)
    cur = np.zeros(28,28)
    for _ in range(100):
        cur += x_from_value[random.randint(0, len(x_from_value))] - x_target[random.randint(0, len(x_target))]
    mean, var = find_mean_var(cur)
    return (cur - mean) / sqrt(var) * eps

def genetic_algorithm(model, pbar, eps=0.5, iterations=100, batch_size=100, randomness=0.01, target=7, from_value=1, init='rand'):
    if init=='rand':
        screens = [create_random_screen(eps=eps) for _ in range(batch_size)]
    else:
        screens = [mean_difference_initialization(target, from_value, eps) for _ in range(batch_size)]

    #num_samples = 1280
    x_train_no_target = get_these_values(from_value)
    y_sample = tf.one_hot(np.array(([target] * len(x_train_no_target))), depth=10)
    scores_timeline = {
        'min' : [],
        'max' : [],
        'mean' : []
    }
    for i in range(iterations):
        scores = [model.evaluate(np.array([elem + s.reshape(28,28,1) for elem in x_train_no_target]), y_sample, verbose=0)[1] for s in screens]
        pbar.update(1)
        pbar.set_description(f"{from_value} -> {target} - Max: {round(max(scores), 2)}, Mean: {round(mean(scores), 2)}, Min: {round(min(scores), 2)}")
        scores_timeline['min'].append(round(min(scores), 2))
        scores_timeline['max'].append(round(max(scores), 2))
        scores_timeline['mean'].append(round(mean(scores), 2))
        
        if i == iterations - 1:
            return screens, scores, scores_timeline
        
        index = selection(scores)
        new_screens = []
        for elem in index:
            new_screens += [make_modifications(screens[elem], randomness=randomness, eps=eps) for i in range(2)]
        screens = new_screens
        del new_screens
    return screens, scores, scores_timeline

def get_two_diff_numbers(number_pairs):
    num1 = random.randint(0,9)
    num2 = random.randint(0,9)
    
    while num1 == num2 and [num1, num2] not in number_pairs:
        num2 = random.randint(0,9)
    return num1, num2

def send_to_github(model_counter, screens, scores, filename, isFirst, number_counter):
    if os.path.exists(f'screens/model{model_counter}'):
        os.system(f'rm -r screens/model{model_counter}')
    os.mkdir(f'screens/model{model_counter}')
    for i in range(len(screens)):
        np.savetxt(f'screens/model{model_counter}/screen_{number_counter}_{i}.txt', screens[i])
    with open(f'screens/model{model_counter}/scores{number_counter}.txt', 'w') as file:
        file.writelines([str(elem)+"\n" for elem in scores])
    os.system(f"git add screens/model{model_counter}")
    os.system("git add model_info.json")
    os.system("git add tracker.txt")
    if isFirst:
        os.system(f"git add {filename}")
    os.system(f"git commit -m \"Adding model{model_counter} info\"")
    os.system("git push origin main")
    os.system(f'rm -r screens/model{model_counter}')
    if isFirst:
        os.system(f"rm -r {filename}")
    return

def run_project():
    try:
        with open('model_info.json', 'r') as file:
            models_info = json.load(file)
    except FileNotFoundError:
        models_info = {
            "CNN" : [],
            "Dense" : [],
            "filename" : [],
            "accuracy" : [],
            "num1" : [],
            "num2" : [],
            "accuracy_timeline" : []
        }
        
    try:
        with open('tracker.txt', 'r') as file:
            starter = int(file.read())
    except:
        starter = -1
        
    counter = 0
    num_iterations=200
    pbar1 = tqdm(total=230*5, desc="Iterating models", position=0)
    pbar2 = tqdm(total=num_iterations, desc="Genetic algorithm", position=1)
        
    for model, cnn_layer_dim, dense_layers, batch_size, filename, acc, filename in iterate_combinations(starter):
        if counter > starter:
            number_pairs = []
            for i in range(5):
                num1, num2 = get_two_diff_numbers(number_pairs)
                number_pairs.append([num1,num2])
                models_info['CNN'].append(cnn_layer_dim)
                models_info['Dense'].append(dense_layers)
                models_info['filename'].append(filename)
                models_info['accuracy'].append(acc)
                models_info['num1'].append(num1)
                models_info['num2'].append(num2)

                with open('tracker.txt', 'w') as file:
                    file.write(str(counter))

                pbar2.reset()
                screens, scores, scores_timeline = genetic_algorithm(model, pbar2, eps=0.5, iterations=num_iterations, batch_size=50, target=num1, from_value=num2)
                models_info['accuracy_timeline'].append(scores_timeline)
                pbar1.update(1)
                
                with open('model_info.json', 'w') as file:
                    json.dump(models_info, file)
                
                send_to_github(counter, screens, scores, filename, i==0, i)
            
        counter += 1
    return
run_project()

Iterating models:   0%|          | 0/1150 [00:00<?, ?it/s]

Genetic algorithm:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1/4
844/844 [==============================] - 1s 1ms/step - loss: 0.3559 - acc: 0.9019 - val_loss: 0.1665 - val_acc: 0.9550
Epoch 2/4
844/844 [==============================] - 1s 1ms/step - loss: 0.1832 - acc: 0.9467 - val_loss: 0.1297 - val_acc: 0.9625
Epoch 3/4
844/844 [==============================] - 1s 980us/step - loss: 0.1392 - acc: 0.9601 - val_loss: 0.1165 - val_acc: 0.9667
Epoch 4/4
844/844 [==============================] - 1s 1ms/step - loss: 0.1144 - acc: 0.9669 - val_loss: 0.1009 - val_acc: 0.9697
INFO:tensorflow:Assets written to: model_configs/model0_savefile/assets
[main 654b2e4] Adding model0 info
 57 files changed, 1459 insertions(+)
 create mode 100644 model_configs/model0_savefile/keras_metadata.pb
 create mode 100644 model_configs/model0_savefile/saved_model.pb
 create mode 100644 model_configs/model0_savefile/variables/variables.data-00000-of-00001
 create mode 100644 model_configs/model0_savefile/variables/variables.index
 create mode 100644 model_info.

To https://github.com/TannerGiddings21/CSI4900.git
   dbc536e..654b2e4  main -> main


[main 5a8ad5f] Adding model0 info
 102 files changed, 1451 insertions(+), 1451 deletions(-)
 rewrite model_info.json (100%)
 rewrite screens/model0/scores.txt (100%)
 delete mode 100644 screens/model0/screen_0_0.txt
 delete mode 100644 screens/model0/screen_0_1.txt
 delete mode 100644 screens/model0/screen_0_10.txt
 delete mode 100644 screens/model0/screen_0_11.txt
 delete mode 100644 screens/model0/screen_0_12.txt
 delete mode 100644 screens/model0/screen_0_13.txt
 delete mode 100644 screens/model0/screen_0_14.txt
 delete mode 100644 screens/model0/screen_0_15.txt
 delete mode 100644 screens/model0/screen_0_16.txt
 delete mode 100644 screens/model0/screen_0_17.txt
 delete mode 100644 screens/model0/screen_0_18.txt
 delete mode 100644 screens/model0/screen_0_19.txt
 delete mode 100644 screens/model0/screen_0_2.txt
 delete mode 100644 screens/model0/screen_0_20.txt
 delete mode 100644 screens/model0/screen_0_21.txt
 delete mode 100644 screens/model0/screen_0_22.txt
 delete mode 100644 s

To https://github.com/TannerGiddings21/CSI4900.git
   654b2e4..5a8ad5f  main -> main
Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7fd5a262c820>
Traceback (most recent call last):
  File "/Users/tannergiddings/opt/anaconda3/lib/python3.9/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [19]:
selection([1]*100, 20)

[80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]